## Notebook for annotations using the geneformer result

In [1]:
import scanpy as sc
import numpy as np
from tqdm.notebook import tqdm
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scvi
import glob

/home/amonell/mambaforge/envs/scvi-env_gpu/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/amonell/mambaforge/envs/scvi-env_gpu/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


### Using the path to the replicate 1 processing, get the Class > Type > Subtype Heirarchy

In [19]:
types = sc.read('/mnt/sata1/Analysis_Alex/timecourse_final/analysis/cleaned/final_celltyped_and_axes.h5ad')
types_ = pd.crosstab(types.obs['Type'], types.obs['Subtype'])
class_ = pd.crosstab(types.obs['Class'], types.obs['Type'])
immuno_ = pd.crosstab(types.obs['Immunocentric_Type'], types.obs['Subtype'])

/home/amonell/mambaforge/envs/scvi-env_gpu/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/amonell/mambaforge/envs/scvi-env_gpu/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


### Create Class > Type > Subtype Heirarchy in replicates based off the GeneFormer result

In [32]:
for path in glob.glob('/mnt/sata1/Analysis_Alex/timecourse_replicates/day*'):
    ad_sp = sc.read(os.path.join(path, 'adatas', '07_geneformer_celltypes.h5ad'))
    #ad_sp.obsm['X_mde'] = scvi.model.utils.mde(ad_sp.obsm['X_scVI_replicates'])

    ad_sp.obs['Subtype'] = ad_sp.obs['celltype_predicted'].values

    type_dictionary = {}
    for i in ad_sp.obs['celltype_predicted'].cat.categories:
        type_dictionary[i] = types_.index.values[np.where(types_[i].values > 0)[0]][0]
    all_types = []
    for k in ad_sp.obs['celltype_predicted'].values:
        all_types.append(type_dictionary.get(k))
    ad_sp.obs['Type'] = all_types

    itype_dictionary = {}
    for i in ad_sp.obs['celltype_predicted'].cat.categories:
        try:
            itype_dictionary[i] = immuno_.index.values[np.where(immuno_[i].values > 0)[0]][0]
        except:
            itype_dictionary[i] = 'None'
    all_itypes = []
    for k in ad_sp.obs['celltype_predicted'].values:
        all_itypes.append(itype_dictionary.get(k))
    ad_sp.obs['Immunocentric_Type'] = all_itypes

    class_dictionary = {}
    for i in np.unique(ad_sp.obs['Type'].values):
        class_dictionary[i] = class_.index.values[np.where(class_[i].values > 0)[0]][0]
    all_classes = []
    for k in ad_sp.obs['Type'].values:
        all_classes.append(class_dictionary.get(k))
    ad_sp.obs['Class'] = all_classes

    sc.pp.neighbors(ad_sp, use_rep='X_scVI_replicates')
    sc.tl.leiden(ad_sp, resolution=2)
    ad_sp.write(os.path.join(path, 'adatas', '08_full_celltypes_and_leiden.h5ad'))
    #sc.pl.embedding(ad_sp, basis='mde', color=['celltype_predicted', 'Type', 'Class', 'Immunocentric_Type'])

/home/amonell/mambaforge/envs/scvi-env_gpu/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/amonell/mambaforge/envs/scvi-env_gpu/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/amonell/mambaforge/envs/scvi-env_gpu/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/amonell/mambaforge/envs/scvi-env_gpu/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
